In [222]:
import torch
import numpy as np
import copy
import os
import cv2
import math
from sklearn import preprocessing

fileadd = "./0_net_ray_marching.pth"
bpnetadd="./1_semanticEmbedding.pth"

# 矩阵最大值最小值
def maxMat(mat):
    maxarr = []
    for i in range(len(mat)):
#         print(mat[i])
        maxarr.append(max(mat[i])) 
    return max(maxarr)

def minMat(mat):
    minarr = []
    for i in range(len(mat)):
        minarr.append(min(mat[i])) 
    return min(minarr)

def normalization(data):
    _range = max(data) - min(data)
    return (data - min(data)) / _range

# 采用方案二
def float2color(zero2one):
    x = zero2one * 256 * 256 * 256
    r = x % 256
    g = ((x - r)/256 % 256)
    b = ((x - r - g * 256)/(256*256) % 256)
    r = round(float(r),2)
    g = round(float(g),2)
    b = round(float(b),2)
    return (r,g,b)

def fixcolor(data):

    return (data,200,150)

def hsv2rgb(h, s, v):
    h = float(h)
    s = float(s)
    v = float(v)
    h60 = h / 60.0
    h60f = math.floor(h60)
    hi = int(h60f) % 6
    f = h60 - h60f
    p = v * (1 - s)
    q = v * (1 - f * s)
    t = v * (1 - (1 - f) * s)
    r, g, b = 0, 0, 0
    if hi == 0: r, g, b = v, t, p
    elif hi == 1: r, g, b = q, v, p
    elif hi == 2: r, g, b = p, v, t
    elif hi == 3: r, g, b = p, q, v
    elif hi == 4: r, g, b = t, p, v
    elif hi == 5: r, g, b = v, p, q
    r, g, b = int(r * 255), int(g * 255), int(b * 255)
    return [r, g, b]




In [223]:
data = torch.load(fileadd)
data["neural_points.points_embeding"].shape
# max(a["neural_points.points_embeding"])
# min(a["neural_points.points_embeding"])

embedding = data["neural_points.points_embeding"][0]
# embedding
# maxnum = maxMat(embedding[0])
# minnum = minMat(embedding[0])
xyz = data["neural_points.xyz"]
print(xyz.shape)
print(embedding.shape)



torch.Size([20550, 3])
torch.Size([20550, 32])


In [224]:
# 对每一列归一化  对vgg网络可视化
print(embedding)
embedT = embedding.T.numpy()
print(embedT.shape)



"""
#线性归一化
embedTnorm = []
for i in range(len(embedT)):
    tem = normalization(copy.deepcopy(embedT[i]))
    embedTnorm.append(tem)
#     print(min(a),max(a))
#     print(a.shape)
embedTnorm = np.array(embedTnorm)

"""

# """
# zscore标准化
embedTnorm = []
for i in range(len(embedT)):
    tem = normalization(copy.deepcopy(embedT[i]))
#     tem = preprocessing.scale(embedT[i])*50+128
    embedTnorm.append(tem)
#     print(min(a),max(a))
#     print(a.shape)
embedTnorm = np.array(embedTnorm)
# """

# embedTnorm = []
# minmax = preprocessing.MinMaxScaler()
# for i in range(len(embedT)):
#     tem = minmax.fit_transform([embedT[i]])
#     embedTnorm.append(tem)
# #     print(min(a),max(a))
# #     print(a.shape)
# embedTnorm = np.array(embedTnorm)


embed = preprocessing.scale(embedTnorm)

print(embedTnorm.shape)
print(embedTnorm)
embed = embedTnorm.T

basepath = "./pointfeature_colorful"
isExists=os.path.exists(basepath)
    # 判断结果
if not isExists:
    # 如果不存在则创建目录
     # 创建目录操作函数
    os.makedirs(basepath) 
    print(basepath+' 创建成功')


for i in range(len(embed[0])):
    mat = []
    for j in range(len(embed)):
        temrow = np.append(xyz[j],fixcolor(embed[j][i]))
        mat.append(temrow)
    
    np.savetxt( os.path.join(basepath,"pointFeature_color{}.txt".format(str(i))), np.array(mat), fmt='%f', delimiter=',')
    print(i)

tensor([[ 3.1833e-02,  6.5631e-01,  9.5582e-01,  ..., -3.3950e-03,
          1.1371e+00,  5.0413e-02],
        [-6.2445e-04,  5.7674e-01,  1.1763e+00,  ..., -3.1546e-03,
          1.5957e+00,  2.9253e-02],
        [ 1.1702e-01,  4.0329e-01,  7.6346e-01,  ..., -1.1105e-03,
          6.4104e-01, -2.3781e-04],
        ...,
        [-7.6880e-04,  9.9284e-01,  7.2517e-01,  ...,  3.7296e-02,
          2.9070e-01, -5.5827e-03],
        [ 1.4753e-02,  8.1437e-01,  8.2889e-01,  ...,  8.9745e-02,
          1.9489e-01, -6.1652e-03],
        [-1.2548e-03, -1.0420e-03,  4.4593e-01,  ..., -4.6876e-03,
          6.2164e-01,  5.4464e-03]])
(32, 20550)
(32, 20550)
[[0.02116549 0.00561014 0.06199167 ... 0.00554096 0.01298006 0.00530804]
 [0.30470788 0.26821482 0.18866561 ... 0.4590437  0.3771946  0.00323022]
 [0.5413591  0.6648008  0.4336652  ... 0.41222608 0.47029328 0.25588772]
 ...
 [0.00317596 0.0033185  0.00453067 ... 0.02730664 0.05840977 0.00240938]
 [0.32502842 0.45510682 0.18431892 ... 0.084945

/home/vr717/anaconda3/envs/pointnerf11/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:236: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  "Numerical issues were encountered "
/home/vr717/anaconda3/envs/pointnerf11/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:255: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  "Numerical issues were encountered "


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31


In [221]:
bpfeature = torch.load(bpnetadd)

print(xyz.shape)

embedT = bpfeature.T.numpy()
print(embedT.shape)
embedTnorm = []
for i in range(len(embedT)):
    tem = normalization(copy.deepcopy(embedT[i]))
#     tem = preprocessing.scale(embedT[i])*70+128
    embedTnorm.append(tem)
#     print(min(a),max(a))
#     print(a.shape)
embedTnorm = np.array(embedTnorm)

embed = embedTnorm.T

"""
float2color
"""
# for i in range(len(embed[0])):
#     mat = []
#     for j in range(len(embed)):
#         temrow = np.append(xyz[j],float2color(embed[j][i]))
#         mat.append(temrow)
    
#     np.savetxt('./bpnetfeature/pointFeatureHSV_{}.txt'.format(str(i)), np.array(mat), fmt='%f', delimiter=',')
#     print(i)


"""
hsv2rgb
"""
# for i in range(len(embed[0])):
#     mat = []
#     for j in range(len(embed)):
#         temrow = np.append(xyz[j],hsv2rgb(embed[j][i],1,1))
#         mat.append(temrow)
    
#     np.savetxt('./bpnetfeature_hsv/pointFeatureHSV_{}.txt'.format(str(i)), np.array(mat), fmt='%f', delimiter=',')
#     print(i)


basepath = "./bpnetfeature_colorful"
isExists=os.path.exists(basepath)
    # 判断结果
if not isExists:
    # 如果不存在则创建目录
     # 创建目录操作函数
    os.makedirs(basepath) 
    print(basepath+' 创建成功')

for i in range(len(embed[0])):
    mat = []
    for j in range(len(embed)):
        temrow = np.append(xyz[j],float2color(embed[j][i]))
        mat.append(temrow)
    
    np.savetxt(os.path.join(basepath,"bpnet_{}.txt".format(str(i))) , np.array(mat), fmt='%f', delimiter=',')
    print(i)

torch.Size([20550, 3])
(96, 20550)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
